## Repetition

Spectral radiance as a function of frequency is described as

$$ L_\nu = \frac{d^4E}{dt\,dA\,d\Omega\,d\nu\,cos(\theta)}=\frac{d^3\Phi}{dA\,d\Omega\,d\nu\,cos(\theta)} \left[\frac{W}{m^2sr\, m}\right]$$

with $\Phi$ being the radiant flux $dE/dt$ and $d\Omega$ the steradian.

Integrated over frequency, this gives [_radiance_](http://en.wikipedia.org/wiki/Radiance):

$$ L = \int L_\nu d\nu = \frac{d^2\Phi}{dA\,d\Omega\,cos(\theta)}$$

I keep the $cos\theta$ as a potential error source.

For radiation emitted by an ideal black body at temperature T, spectral radiance is governed by [Planck's law](http://en.wikipedia.org/wiki/Planck%27s_law), while the integral of radiance over the hemisphere into which it radiates, in W/m2, is governed by the [Stefan-Boltzmann law](http://en.wikipedia.org/wiki/Stefan-Boltzmann_law).

Planck's law describes the emitted radiation as:

$$ B_\nu(T) = \frac{2h\nu^3}{c^2}\frac{1}{exp\left(\frac{hc}{\lambda k_BT}\right)-1} $$

Marc:
>Keep in mind that “absolute” radiances have an arbitrary constant associated with them, having to do with the fact that we don’t measure absolute spectral response, just relative spectral response.
 
>I was a little sloppy in not normalizing the channel 8 and 9 spectral response curves to their peak values as was done for the other channels.  However, the arbitrary spectral response scale factor present in all channels is irrelevant when one converts back to brightness temperature.  Any absolute radiance is not quantitatively meaningful without the associated spectral response curve for that channel.  See attached document and spreadsheet.

Tim:
>Because the spectral response curve $F(\nu)$ is relative, the only unambiguous definition of integrated radiance is given by:
$$R(ChX, T) =  \frac{\int{B(\nu, T)\cdot F(\nu, ChX)\cdot d\nu}}{\int{F(\nu, ChX)\cdot d\nu}}$$
where $B(\nu, T)$ is the spectral Planck function, and $ChX$ refers to channel $X$. 
The normalization is then by the integral of the spectral response curve, not by an arbitrary number to make the peak unity.
This has the units of spectral Planck $\frac{W}{cm^2\cdot sr\cdot cm}$, rather than spectrally integrated Planck $\frac{W}{cm^2\cdot sr}$, which is irritating. The misunderstanding below is an example of why it should be done this way!

### are we close to Rayleigh-Jeans criterium?
For that, $h\nu << k_BT$ has to be true.
I will test if the value of $\frac{h\nu}{k_BT}$ is always much smaller than 1.

In [1]:
from scipy.constants import k as k_B
from scipy.constants import h, c

In [2]:
wavelengths = np.array([8, 20, 50, 100, 200, 300, 400])

In [3]:
temps = np.array([20,100,200,300,400])

In [4]:
import pandas as pd
df = pd.DataFrame(index=wavelengths)

In [5]:
for T in temps:
    ratio = h*c / wavelengths / 1e-6 / (k_B *T) # divide by 1e-6 to make up for waves being microns
    df[T] = ratio

In [6]:
df.index.name='Wavelengths[mu]'
df.columns.name='Temperatures[K]'

In [7]:
df

Temperatures[K],20,100,200,300,400
Wavelengths[mu],,,,,
8,89.923585,17.984717,8.992358,5.994906,4.496179
20,35.969434,7.193887,3.596943,2.397962,1.798472
50,14.387774,2.877555,1.438777,0.959185,0.719389
100,7.193887,1.438777,0.719389,0.479592,0.359694
200,3.596943,0.719389,0.359694,0.239796,0.179847
300,2.397962,0.479592,0.239796,0.159864,0.119898
400,1.798472,0.359694,0.179847,0.119898,0.089924


In [8]:
%matplotlib widget

In [17]:
ax = df.plot(logy=True)
ax.set_xlabel("Wavelength [micron]")
ax.set_title('Rayleigh-Jeans check for set of temperatures')

FigureCanvasNbAgg()

Text(0.5, 1.0, 'Rayleigh-Jeans check for set of temperatures')

The fact that the data are mostly above 1 means we can not use simplified laws for brightness temperature $T_b$.

In [10]:
from astropy.modeling.blackbody import blackbody_lambda, blackbody_nu

In [12]:
wavelengths = np.logspace(0, 3, 100) * u.micron

In [13]:
wavelengths

<Quantity [   1.        ,    1.07226722,    1.149757  ,    1.23284674,
              1.32194115,    1.41747416,    1.51991108,    1.62975083,
              1.7475284 ,    1.87381742,    2.009233  ,    2.15443469,
              2.3101297 ,    2.47707636,    2.65608778,    2.84803587,
              3.05385551,    3.27454916,    3.51119173,    3.76493581,
              4.03701726,    4.32876128,    4.64158883,    4.97702356,
              5.33669923,    5.72236766,    6.13590727,    6.57933225,
              7.05480231,    7.56463328,    8.11130831,    8.69749003,
              9.32603347,   10.        ,   10.72267222,   11.49756995,
             12.32846739,   13.21941148,   14.17474163,   15.19911083,
             16.29750835,   17.475284  ,   18.73817423,   20.09233003,
             21.5443469 ,   23.101297  ,   24.77076356,   26.56087783,
             28.48035868,   30.53855509,   32.74549163,   35.11191734,
             37.64935807,   40.37017259,   43.28761281,   46.41588834,
      

In [14]:
temp = 40 * u.K

In [15]:
flux_lambda = blackbody_lambda(wavelengths, temp)

In [19]:
from astropy.visualization import quantity_support

In [55]:
fig, ax = plt.subplots(constrained_layout=True)
ax.plot(wavelengths, flux_lambda)

FigureCanvasNbAgg()

In [86]:
from astropy.visualization import quantity_support
from astropy.modeling.blackbody import blackbody_lambda

class DivBandpasses():
    def __init__(self):
        self.fname = "../diviner/data/Diviner_R_to_T_8-4-2009.xls"
        df = pd.read_excel(fname, 'Bandpasses', header=3, skiprows=4)
        df.drop([0,1], inplace=True)
        df.columns = ' '.join(['w{0} ch{0}'.format(i) for i in range(3,10)]).split()
        self.df = df
        for ch in range(3, 10):
            setattr(self, f"ch{ch}", self.get_channel_X(ch))
        self.wavelengths = np.logspace(0, 3, 100) * u.micron
            
    def get_channel_X(self, X):
        tmp = self.df[[f'w{X}', f'ch{X}']].dropna()
        return tmp.set_index(f'w{X}')
    
    def plot_all(self, ax=None):
        if ax is None:
            fig, ax = plt.subplots(constrained_layout=True)
        for ch in range(3, 10):
            getattr(self, f"ch{ch}").plot(ax=ax, secondary_y=True)
    
    def plot_planck_with_bandpasses(self, temps):
        fig, ax = plt.subplots(constrained_layout=True)
        for temp in temps:
            fluxes = blackbody_lambda(self.wavelengths, temp)
            ax.plot(self.wavelengths, fluxes, label=temp)
        ax.legend()
        self.plot_all(ax)


In [87]:
bandpasses = DivBandpasses()

In [88]:
temps = [25, 30, 35, 40] * u.K

In [89]:
bandpasses.plot_planck_with_bandpasses(temps)

FigureCanvasNbAgg()

In [66]:
fig, ax = plt.subplots(constrained_layout=True)
ax.plot(wavelengths, flux_lambda, label='40 K')
ax.legend()
bandpasses.ch8.plot(ax=ax, secondary_y=True)

FigureCanvasNbAgg()